# *Model:* ARMA
# *Stock Data:* NSEI
#  *Name: *Abbas Amjherawala**

In [ ]:
!pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from tqdm import tqdm
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta, timezone
from time import time

# **Import Data and calculate Returns**
> **plots:** *prices of stocks over time and returns of stocks over time*

In [ ]:
tickerSymbol = '^NSEI'
data = yf.Ticker(tickerSymbol)
data

In [ ]:
prices = data.history(start='2022-03-03', end='2023-03-03').Close
prices

In [ ]:
returns = prices.pct_change().dropna()
returns

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(prices)
plt.ylabel('Closing Prices', fontsize=20)
plt.title('Stock Rate over Time', fontsize=20)

dates = pd.date_range(start='2022-03-03', end='2023-03-03', freq='M')
for date in dates:
    plt.axvline(date, color='k', linestyle='--', alpha=0.2)
plt.axhline(prices.mean(), color='r', alpha=0.2, linestyle='--')

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(returns)
plt.ylabel('Returns', fontsize=20)
plt.title('Stock Returns over Time', fontsize=20)

dates = pd.date_range(start='2022-03-03', end='2023-03-03', freq='M')
for date in dates:
    plt.axvline(date, color='k', linestyle='--', alpha=0.2)
plt.axhline(returns.mean(), color='r', alpha=0.2, linestyle='--')

# ACF for MA
> No proper identifications captured

In [ ]:
plot_acf(returns)
plt.show()

# PACF for AR
> No proper identification captured

In [ ]:
plot_pacf(returns, method='ywm')
plt.show()

# Train and Test data splits
1. divide price dataframe
2. train ARIMA model on train set
3. test order of ARIMA model among: (0,0,0), (4,0,4), (4,0,6) and (6,0,6)
4. predicting test data on ARIMA order (5,0,4), best
5. plotting original vs predicted data
6. calculating acuuracy: mae, mse, mape

In [ ]:
p_train_date='2023-01-02'
p_train_data = prices[:p_train_date]
p_train_data.index = pd.to_datetime(p_train_data.index)
p_train_data

In [ ]:
p_test_date='2023-01-03'
p_test_data = prices[p_test_date :]
p_test_data.index = pd.to_datetime(p_test_data.index)
p_test_data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model = ARIMA(p_train_data, order=(0, 0, 0))
fitted = model.fit()

next_day_price = fitted.forecast()
predicted_price = next_day_price.values[0]
print("Predicted price for the 2023-01-03:", predicted_price)
print("Original price for the 2023-01-03:", p_test_data[0])
print("Difference between original and predicted price:", p_test_data[0] - predicted_price)

In [ ]:
print(fitted.summary())

In [ ]:
model = ARIMA(p_train_data, order=(4, 0, 4))
fitted = model.fit()

next_day_price = fitted.forecast()
predicted_price = next_day_price.values[0]
print("Predicted price for the 2023-01-03:", predicted_price)
print("Original price for the 2023-01-03:", p_test_data[0])
print("Difference between original and predicted price:", p_test_data[0] - predicted_price)

In [ ]:
model = ARIMA(p_train_data, order=(4, 0, 6))
fitted = model.fit()

next_day_price = fitted.forecast()
predicted_price = next_day_price.values[0]
print("Predicted price for the 2023-01-03:", predicted_price)
print("Original price for the 2023-01-03:", p_test_data[0])
print("Difference between original and predicted price:", p_test_data[0] - predicted_price)

In [ ]:
model = ARIMA(p_train_data, order=(6, 0, 6))
fitted = model.fit()

next_day_price = fitted.forecast()
predicted_price = next_day_price.values[0]
print("Predicted price for the 2023-01-03:", predicted_price)
print("Original price for the 2023-01-03:", p_test_data[0])
print("Difference between original and predicted price:", p_test_data[0] - predicted_price)

In [ ]:
print(fitted.summary())

In [ ]:
i=0
total_profit=0
total_loss=0
predictions=[]
for check in p_test_data:
    model = ARIMA(p_train_data, order=(5, 0, 4))
    fitted = model.fit()

    next_day_price = fitted.forecast()
    predicted_price = next_day_price.values[0]
    print(f"Predicted price for the {i+1} day:", predicted_price)
    predictions.append(predicted_price)
    print(f"Original price for the {i+1} day:", check)
    pr_ls= check - predicted_price
    if pr_ls<0:
        print("Loss of:", abs(pr_ls))
        total_loss+=pr_ls
    elif pr_ls==0:
        print("No Profit and No Loss")
    else:
        print("Profit of:", abs(pr_ls))
        total_profit+=pr_ls
    print("\n")
    p_train_data.loc[len(p_train_data)+i] = p_test_data.values[i]
    i=i+1

In [ ]:
print("From January to March")
print("total profit of Rs.",total_profit)
print("Total loss of Rs.",total_loss)

In [ ]:
predict = pd.DataFrame(predictions, index=p_test_data.index)
plt.figure(figsize=(10,4))

plt.plot(prices)
plt.plot(predict)
plt.legend(('Original Price', 'Predicted Price'), fontsize=16)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [ ]:
residuals = p_test_data - predictions
print("The accuracy of the ARIMA model for the test/train data is as stated as below:")
print('Mean Absolute Error:', mean_absolute_error(p_test_data, predictions))
print('Mean Squared Error:', mean_squared_error(p_test_data, predictions))
print('Root Mean Squared Error:', np.sqrt(np.mean(residuals**2)))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(p_test_data, predictions))

# Predicting Stck Price for tomorrow (04-03-2023) using ARIMA

In [ ]:
import datetime
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365)
prices = data.history(start=start_date, end=end_date).Close
prices

In [ ]:
model = ARIMA(prices, order=(0, 0, 0))
fitted = model.fit()

next_day_price = fitted.forecast()
predicted_price = next_day_price.values[0]
print(f"Predicted price for the {end_date}, i.e. tomorrow:", predicted_price)

# Function to Analyse Stock for average and deviation returns
> User Inputs:
* Provide start date input of the form: dd-mm-yyyy 
* Provide end date input of the form: dd-mm-yyyy 
* Provide Retrun Period Weeks
* Provide Minimum Avg Return
* Provide Maximum Deviation Return
* Calculate future price of stock using ARIMA 5,0,4

In [ ]:
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
from pandas_datareader._utils import RemoteDataError

In [ ]:
def plot_stock_trend_and_returns(prices, titles, start_date, end_date, all_returns):
    
    plt.figure(figsize=(10,6))
    
    plt.subplot(2,1,1)
    plt.plot(prices)
    plt.title(titles[0], fontsize=16)
    plt.ylabel('Price (Rs.)', fontsize=14)
    
    plt.subplot(2,1,2)
    plt.plot(all_returns[0], all_returns[1], color='g')
    plt.title(titles[1], fontsize=16)
    plt.ylabel('Return (Rs.)', fontsize=14)
    plt.axhline(0, color='k', linestyle='--')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def perform_analysis_for_stock(prices, start_date, end_date, return_period_weeks, verbose=False):    
    pct_return_after_period = []
    buy_dates = []

    for buy_date, buy_price in prices.iteritems():
        sell_date = buy_date + timedelta(weeks=return_period_weeks)
        
        try:
            sell_price = prices[prices.index == sell_date].iloc[0]
        except IndexError:
            continue
        
        pct_return = (sell_price - buy_price)/buy_price
        if pct_return>0:
            pct_return_after_period.append(pct_return)
            buy_dates.append(buy_date)
        
            if verbose:
                print('Date Buy: %s, Price Buy: %s'%(buy_date,round(buy_price,2)))
                print('Date Sell: %s, Price Sell: %s'%(sell_date,round(sell_price,2)))
                print('Return: %s%%'%round(pct_return*100,1))
                print('-------------------')
    
    if len(pct_return_after_period) == 0:
        return -np.inf, np.inf, None
    
    return np.mean(pct_return_after_period), np.std(pct_return_after_period), [buy_dates, pct_return_after_period]

In [ ]:
from datetime import datetime

input_str=input("Provide start date input of the form: dd-mm-yyyy")
input_s_dt = datetime.strptime(input_str, '%d-%m-%Y')
input_str=input("Provide end date input of the form: dd-mm-yyyy")
input_e_dt = datetime.strptime(input_str, '%d-%m-%Y')
start_date, end_date = datetime(input_s_dt.year, input_s_dt.month, input_s_dt.day), datetime(input_e_dt.year, input_e_dt.month, input_e_dt.day)
return_period_weeks = int(input("Provide Retrun Period Weeks:"))
min_avg_return  = float(input("Provide Minimum Avg Return:"))
max_dev_return = float(input("Provide Maximum Deviation Return:"))

now = datetime.now()
if end_date > now:
    prices = data.history(start=start_date, end=now).Close
else: prices = data.history(start=start_date, end=end_date).Close
prices

In [ ]:
import datetime
tz = 'Asia/Kolkata'
while now < end_date:
    model = ARIMA(prices, order=(5, 0, 4))
    fitted = model.fit()
    next_day_price = fitted.forecast()
    predicted_price = next_day_price.values[0]
    ts = pd.Timestamp(now).tz_localize('UTC').tz_convert(tz)
    new_data = pd.Series([predicted_price], index=[ts])
    prices = pd.concat([prices, new_data])
    now = now + datetime.timedelta(days=1)

In [ ]:
avg_return, dev_return, all_returns = perform_analysis_for_stock(prices, start_date, end_date, return_period_weeks,True)
title_price = 'NSEI'
title_return = 'Avg Return: %s%% | Dev Return: %s%%'%(round(100*avg_return,2), round(100*dev_return,2))
plot_stock_trend_and_returns(prices, [title_price, title_return], start_date, end_date, all_returns)
if avg_return > min_avg_return and dev_return < max_dev_return:
    print("The Avg Return and Dev Return were High!")
else:
    print("The Avg Return and Dev Return were too Low!")

# Function to simulate user amount and give return *based on user inputs*

In [ ]:
def run_simulation(returns, prices, amt, thresh, order=(5,0,1), verbose=True, plot=True):
    curr_holding = False
    events_list = []
    init_amt = amt
    #go through dates
    for date, r in tqdm (returns.iloc[14:].items(), total=len(returns.iloc[14:])):
        #if you're currently holding the stock, sell it
        if curr_holding:
            sell_price = prices.loc[date]
            curr_holding=False
            ret = (sell_price-buy_price)/buy_price
            amt *= (1+ret)
            events_list.append(('s', date, ret))
            
            if verbose:
                print('Sold at $%s'%sell_price)
                print('Predicted Return: %s'%round(pred,4))
                print('Actual Return: %s'%(round(ret, 4)))
                print('=======================================')
            continue

        #get data til just before current date
        curr_data = returns[:date]
        model = ARIMA(curr_data, order=order).fit()
        pred = model.forecast().values[0]

        #if you predict a high enough return and not holding, buy stock
        if (not curr_holding) and pred > thresh:
            curr_holding = True
            buy_price = prices.loc[date]
            events_list.append(('b', date))
            if verbose:
                print('Bought at $%s'%buy_price)
                
    if verbose:
        print('Total Amount: $%s'%round(amt,2))
        
    #graph
    if plot:
        plt.figure(figsize=(10,4))
        plt.plot(prices[14:])

        y_lims = (int(prices.min()*.95), int(prices.max()*1.05))
        shaded_y_lims = int(prices.min()*.5), int(prices.max()*1.5)

        for idx, event in enumerate(events_list):
            plt.axvline(event[1], color='k', linestyle='--', alpha=0.4)
            if event[0] == 's':
                color = 'green' if event[2] > 0 else 'red'
                plt.fill_betweenx(range(*shaded_y_lims), 
                                  event[1], events_list[idx-1][1], color=color, alpha=0.1)

        tot_return = round(100*(amt / init_amt - 1), 2)
        tot_return = str(tot_return) + '%'
        plt.title("%s Price Data\nThresh=%s\nTotal Amt: $%s\nTotal Return: %s"%(tickerSymbol, thresh, round(amt,2), tot_return), fontsize=20)
        plt.ylim(*y_lims)
        plt.show()
    
    return amt

In [ ]:
from datetime import datetime
input_str=input("Provide start date input of the form: dd-mm-yyyy")
input_s_dt = datetime.strptime(input_str, '%d-%m-%Y')
input_str=input("Provide end date input of the form: dd-mm-yyyy")
input_e_dt = datetime.strptime(input_str, '%d-%m-%Y')
start_date, end_date = datetime(input_s_dt.year, input_s_dt.month, input_s_dt.day), datetime(input_e_dt.year, input_e_dt.month, input_e_dt.day)
prices = data.history(start=start_date, end=end_date).Close

input_amt=float(input("Provide Amount:"))
input_th=float(input("Provide Threshold:"))
returns = prices.pct_change().dropna()
run_simulation(returns, prices, input_amt, input_th)